# Google Colab Setup for LLM Fine-tuning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/farhan1301/Fine-Tuning-LLM/blob/main/colab/00_colab_setup.ipynb)

This notebook sets up Google Colab for fine-tuning Llama 3.2 1B.

**GPU:** Make sure you have GPU enabled!
- Go to: Runtime → Change runtime type → Hardware accelerator → T4 GPU

**Estimated Time:** 5-10 minutes for setup

## 1. Check GPU Availability

In [ ]:
import torch

print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → T4 GPU")

## 2. Mount Google Drive (to save models)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/LLM_Finetuning'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/data", exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/models", exist_ok=True)

print(f"✓ Project directory: {PROJECT_DIR}")

## 3. Install Required Packages

In [ ]:
%%capture
# Install NumPy first to avoid binary incompatibility
!pip install -q 'numpy<2.0'
# Install packages (suppress output)
!pip install -q -U transformers>=4.40.0
!pip install -q -U peft>=0.10.0
!pip install -q -U trl>=0.8.0
!pip install -q -U bitsandbytes>=0.41.0
!pip install -q -U accelerate>=0.28.0
!pip install -q -U datasets>=2.14.0
!pip install -q -U pydantic>=2.5.0
!pip install -q -U anthropic
!pip install -q -U gradio

print("✓ All packages installed")

## 4. Verify Installation

In [ ]:
import transformers
import peft
import trl
import torch

print("Package Versions:")
print(f"  PyTorch: {torch.__version__}")
print(f"  Transformers: {transformers.__version__}")
print(f"  PEFT: {peft.__version__}")
print(f"  TRL: {trl.__version__}")
print(f"\n✓ All packages loaded successfully")

## 5. Set Up API Keys

**Important:** Add your API keys to Colab Secrets:
1. Click the key icon (🔑) in left sidebar
2. Add secrets:
   - `HUGGINGFACE_TOKEN` - from https://huggingface.co/settings/tokens
   - `ANTHROPIC_API_KEY` - from https://console.anthropic.com/

Or use the form below:

In [ ]:
import os
from google.colab import userdata

try:
    # Try to get from Colab secrets
    HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    print("✓ API keys loaded from Colab secrets")
except:
    # Manual input if secrets not set
    print("⚠️ Colab secrets not found. Enter manually:")
    from getpass import getpass
    HUGGINGFACE_TOKEN = getpass("Enter Hugging Face token: ")
    ANTHROPIC_API_KEY = getpass("Enter Anthropic API key: ")

# Set environment variables
os.environ['HUGGINGFACE_TOKEN'] = HUGGINGFACE_TOKEN
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

print("✓ API keys configured")

## 6. Login to Hugging Face

In [ ]:
from huggingface_hub import login

login(token=HUGGINGFACE_TOKEN)
print("✓ Logged in to Hugging Face")
print("\n⚠️ Make sure you've accepted the Llama 3.2 license:")
print("   https://huggingface.co/meta-llama/Llama-3.2-1B")

## 7. Test Model Loading

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

print("Testing model loading with 4-bit quantization...")

# 4-bit config (GPU optimized)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load small test
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

print(f"✓ Model loaded successfully")
print(f"  Memory: {model.get_memory_footprint() / 1e9:.2f} GB")
print(f"  Device: {next(model.parameters()).device}")

## 8. Setup Complete!

### What's Ready:
- ✓ GPU enabled and verified
- ✓ Google Drive mounted
- ✓ All packages installed
- ✓ API keys configured
- ✓ Hugging Face authenticated
- ✓ Model loading tested

### Next Steps:

**Continue with these Colab notebooks:**
1. `01_colab_data_generation.ipynb` - Generate training data (2-3 hrs)
2. `02_colab_training.ipynb` - Train model (1-2 hrs on T4 GPU)
3. `03_colab_inference.ipynb` - Test and use the model

### Important Notes:
- **Save frequently**: Colab disconnects after 12 hours or inactivity
- **Models saved to**: Google Drive → `LLM_Finetuning/models/`
- **Data saved to**: Google Drive → `LLM_Finetuning/data/`
- **Free GPU limit**: ~15 hours/day on T4 GPU

### GPU vs CPU Training Time:
- **CPU (local)**: 12-24 hours
- **T4 GPU (Colab)**: 1-2 hours ⚡

**You'll save ~10-20 hours using Colab!**